In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install flax

In [ ]:
!pip install transformers[flax]

In [ ]:
import jax
jax.devices()
import requests
from PIL import Image
import io
import json
import base64
import logging

from flask import Flask, request, jsonify, abort
from flask_ngrok import run_with_ngrok

from transformers import ViTFeatureExtractor, AutoTokenizer, FlaxVisionEncoderDecoderModel

loc = "ydshieh/vit-gpt2-coco-en"

feature_extractor = ViTFeatureExtractor.from_pretrained(loc)
tokenizer = AutoTokenizer.from_pretrained(loc)
model = FlaxVisionEncoderDecoderModel.from_pretrained(loc)

def generate_step(img):
    pixel_values = feature_extractor(images=img, return_tensors="np").pixel_values
    output_ids = model.generate(pixel_values, max_length=16, num_beams=4).sequences
    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds[0]

def vidi(image):
    return generate_step(image)


app = Flask(__name__)
app.logger.setLevel(logging.DEBUG)
run_with_ngrok(app)

@app.route("/describe", methods=['POST'])
def describe():
    # print(request.json)
    if not request.json or 'image' not in request.json:
        abort(400)

    # get the base64 encoded string
    im_b64 = request.json['image']

    # convert it into bytes
    img_bytes = base64.b64decode(im_b64.encode('utf-8'))

    # convert bytes data to PIL Image object
    img = Image.open(io.BytesIO(img_bytes))

    output = vidi(img)

    result_dict = {'output': output}
    return result_dict


if __name__ == "__main__":
    app.run()